In [43]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import csv
import pickle

In [44]:
with open('../data/ip_2_mac_dict.pickle', 'r') as f:
    ip_2_mac_dict = pickle.load(f)
    
with open('../data/host_data_dict.pickle', 'r') as f:
    host_data_dict = pickle.load(f)

In [45]:
mac_2_manuf = {}
with open('../data/mac-manuf.csv') as f:
    for line in f:
        tmp = line.split(',')
        mac_2_manuf[tmp[0].lower()] = tmp[1].split('\n')[0]

In [46]:
import glob
dns_fnames = glob.glob("../data/tshark_20170207_20170212/*.csv")

for dns_fname in dns_fnames:
    print "Processing file", dns_fname
    dns_data = []
    out_str = ''
    with open(dns_fname, 'r') as f:
        ctr = 0
        for line in f:
            tmp = line.split('|')
            if len(tmp) > 2:
                tmp[-1] = tmp[-1].split('\n')[0]
                ip_addr_str = tmp[2]
                    
                if ip_addr_str in host_data_dict:
                    mac_addr_str = host_data_dict[ip_addr_str][0]
                    if mac_addr_str == '' and ip_addr_str in ip_2_mac_dict:
                        mac_addr_str = list(list(ip_2_mac_dict[ip_addr_str])[0])[0]
                    trimmed_str = []
                    for elem in mac_addr_str.split(':')[:3]:
                        if len(elem) == 1:
                            trimmed_str.append('0'+elem.lower())
                        else:
                            trimmed_str.append(elem.lower())
                            
                    mac_addr_str_trimmed = ':'.join([str(x) for x in trimmed_str])
                    
                    if mac_addr_str_trimmed in mac_2_manuf:
                        manuf = mac_2_manuf[mac_addr_str_trimmed]
                    else:
                        manuf = ''
                    
                    tmp += list(host_data_dict[ip_addr_str])+[manuf]
                else:
                     tmp += ['','','', '','','', '']     
                dns_data.append(tuple(tmp))
                out_str += ','.join([str(x) for x in tmp])+'\n'
        out_str = out_str[:-1]
        
        
    
    dump_fname = '../data/tshark_20170207_20170212_consolidated/'+dns_fname.split('.csv')[0].split('data/tshark_20170207_20170212/')[1]+'.csv'
    print "Dumping file:", dump_fname
    with open(dump_fname, 'w') as f:
        f.write(out_str)
    break
    
        

Processing file ../data/tshark_20170207_20170212/split_hour_00118_20170211215900.csv
Dumping file: ../data/tshark_20170207_20170212_consolidated/split_hour_00118_20170211215900.csv


In [35]:
mac_2_manuf['00:0c:29']

'Vmware'